In [94]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR 
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputRegressor

import sys
import os
from tqdm import tqdm

RAND = 10
N_FOLDS = 3

backend_path = os.path.abspath('../backend')
sys.path.append(backend_path)
from get_metrics import get_metrics_regression, get_metrics_classification
from check_overfitting import check_overfitting

In [51]:
# Чтение DataFrame df в файл data/df.csv
df = pd.read_csv('../data/df.csv')

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94354 entries, 0 to 94353
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   milliseconds    94354 non-null  int64  
 1   place           94354 non-null  object 
 2   status          94354 non-null  object 
 3   tsunami         94354 non-null  int64  
 4   significance    94354 non-null  int64  
 5   data_type       94354 non-null  object 
 6   magnitude       94354 non-null  float64
 7   country         94354 non-null  object 
 8   longitude       94354 non-null  float64
 9   latitude        94354 non-null  float64
 10  depth           94354 non-null  float64
 11  datetime        94354 non-null  object 
 12  timezone        94354 non-null  object 
 13  magnitude_bins  94354 non-null  object 
 14  year            94354 non-null  int64  
 15  month           94354 non-null  int64  
 16  day             94354 non-null  int64  
 17  hour            94354 non-null 

In [53]:
# создаем LabelEncoder для кодирования категориальных значений
#le = LabelEncoder()
# кодирование колонки 'country'
#le.fit(df['country'])  # обучаем энкодер на df
#df['country'] = le.fit_transform(df['country'])

# разделение данных train_test_split

In [54]:
# признаки
X = df[['milliseconds', 'significance', 'country', 'depth', 'year', 'month', 'day', 'hour', 'minute', 'second']]

# целевые переменные
y = df[['magnitude', 'longitude', 'latitude']]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=RAND)


In [55]:
# создаем LabelEncoder для кодирования категориальных значений
le = LabelEncoder()

# обучаем le на тренировочных и тестовых данных
le.fit(np.concatenate((X_train['country'], X_test['country'])))

# кодирование колонки 'country' для обучения моделей
X_train['country'] = le.transform(X_train['country'])
X_test['country'] = le.transform(X_test['country'])

# Baseline 

## RandomForestRegressor

In [56]:
# модель RandomForestRegressor
rfr = RandomForestRegressor(n_jobs=-1, random_state=RAND)
# обучаем модель
rfr.fit(X_train, y_train)
# предсказания на test 
y_pred_rfr = rfr.predict(X_test)

In [57]:
# проверка на переобучение модели RandomForestRegressor
check_overfitting(rfr, X_train, y_train, X_test, y_test, mean_squared_error)

mean_squared_error train: 40.710
mean_squared_error test: 268.426
delta = 84.8 %


In [58]:
# просмотр метрик модели RandomForestRegressor обученной на train
metrics = get_metrics_regression(y_test,
                                 y_pred = y_pred_rfr,
                                 X_test = X_test,
                                 name='RandomForestRegressor_Baseline')
metrics

,model,MAE,MSE,RMSE,RMSLE,R2 adjusted,MPE_%,MAPE_%,WAPE_%
0,RandomForestRegressor_Baseline,6.336102,268.425542,16.383697,None,0.888419,-inf,inf,NaN


In [59]:
# n_estimators_range = np.arange(100, 1000, 100)
# train_errors = []
# test_errors = []
# for n_estimators in n_estimators_range:
#     # Обучите модель RandomForestRegressor
#     rf = RandomForestRegressor(n_estimators=n_estimators)
#     rf.fit(X_train, y_train)

#     # Сделайте прогнозы на обучающей и тестовой выборках
#     y_train_pred = rf.predict(X_train)
#     y_test_pred = rf.predict(X_test)

#     # Рассчитайте ошибки
#     train_error = mean_squared_error(y_train, y_train_pred)
#     test_error = mean_squared_error(y_test, y_test_pred)

#     # Добавьте ошибки в списки
#     train_errors.append(train_error)
#     test_errors.append(test_error)


In [60]:
# plt.plot(n_estimators_range, train_errors, label='Ошибка обучения')
# plt.plot(n_estimators_range, test_errors, label='Ошибка тестирования')
# plt.xlabel('Количество деревьев (n_estimators)')
# plt.ylabel('Средняя квадратичная ошибка')
# plt.title('Зависимость ошибки от количества деревьев')
# plt.legend()
# plt.show()

## DecisionTreeRegressor

In [61]:
# модель DecisionTreeRegressor
dtr = DecisionTreeRegressor(random_state=RAND)
# обучаем модель
dtr.fit(X_train, y_train)
# предсказания на test 
y_pred_dtr = dtr.predict(X_test)

In [62]:
# проверка на переобучение модели DecisionTreeRegressor
check_overfitting(dtr, X_train, y_train, X_test, y_test, mean_squared_error)

mean_squared_error train: 0.000
mean_squared_error test: 513.967
delta = 100.0 %


In [63]:
# просмотр метрик модели DecisionTreeRegressor обученной на train
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred = y_pred_dtr,
                           X_test = X_test,
                           name='DecisionTreeRegressor_Baseline')])
metrics

,model,MAE,MSE,RMSE,RMSLE,R2 adjusted,MPE_%,MAPE_%,WAPE_%
0,RandomForestRegressor_Baseline,6.336102,268.425542,16.383697,None,0.888419,-inf,inf,NaN
0,DecisionTreeRegressor_Baseline,7.378031,513.967447,22.670850,None,0.781858,-inf,inf,NaN


## GradientBoostingRegressor

In [64]:
# модель GradientBoostingRegressor
gbr = MultiOutputRegressor(GradientBoostingRegressor(random_state=RAND)) 
# обучаем модель
gbr.fit(X_train, y_train)
# предсказания на test 
y_pred_gbr = gbr.predict(X_test)


In [65]:
# проверка на переобучение модели GradientBoostingRegressor
check_overfitting(gbr, X_train, y_train, X_test, y_test, mean_squared_error)

mean_squared_error train: 434.538
mean_squared_error test: 429.918
delta = 1.1 %


In [66]:
# просмотр метрик модели GradientBoostingRegressor обученной на train
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred = y_pred_gbr,
                           X_test = X_test,
                           name='GradientBoostingRegressor_Baseline')])
metrics

,model,MAE,MSE,RMSE,RMSLE,R2 adjusted,MPE_%,MAPE_%,WAPE_%
0,RandomForestRegressor_Baseline,6.336102,268.425542,16.383697,None,0.888419,-inf,inf,NaN
0,DecisionTreeRegressor_Baseline,7.378031,513.967447,22.670850,None,0.781858,-inf,inf,NaN
0,GradientBoostingRegressor_Baseline,9.747325,429.918285,20.734471,None,0.827049,-inf,inf,NaN


## SVR

In [67]:
# # модель SVR
# svr = SVR(kernel='rbf', C=1, epsilon=0.1)
# # мульти-регрессор с параллельной обработкой
# morsvr = MultiOutputRegressor(svr, n_jobs=-1)
# # обучаем модель
# morsvr.fit(X_train, y_train)
# # предсказания на test 
# y_pred_svr = morsvr.predict(X_test)

In [68]:
# # проверка на переобучение модели SVR
# check_overfitting(morsvr, X_train, y_train, X_test, y_test, mean_squared_error)

In [69]:
# # просмотр метрик модели SVR обученной на train
# metrics = pd.concat([
#     metrics,
#     get_metrics_regression(y_test,
#                            y_pred = morsvr.predict(X_test),
#                            X_test = X_test,
#                            name='SVR_Baseline')])
# metrics

# RandomizedSearchCV  

## RandomForestRegressor

In [109]:
# сетка параметров
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'max_features': [0.5, 0.75, 1],
}

# расчет количества моделей
num_models = math.prod(len(values) for values in param_grid.values())

print(f"Количество моделей для обучения: {num_models}")

Количество моделей для обучения: 81


In [110]:
# модель RandomForestRegressor
rfr = RandomForestRegressor(random_state=RAND, n_jobs=-1)

# модель RandomizedSearchCV и обучение по сетке
rsCV = RandomizedSearchCV(rfr, param_grid, cv=3, n_iter=20, n_jobs=-1, random_state=RAND)
rsCV.fit(X_train, y_train)

RandomizedSearchCV(cv=3,
                   estimator=RandomForestRegressor(n_jobs=-1, random_state=10),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'max_depth': [5, 10, 20],
                                        'max_features': [0.5, 0.75, 1],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 500, 1000]},
                   random_state=10)

In [111]:
# просмотр параметров
best_params_rfr = rsCV.best_params_
print(best_params_rfr)
print(rsCV.best_score_)

{'n_estimators': 1000, 'min_samples_split': 2, 'max_features': 0.75, 'max_depth': 20}
0.8743852263418223


In [112]:
# модель RandomForestRegressor лучшими параметрами
rfr_best_params = RandomForestRegressor(**best_params_rfr, random_state=RAND)
# обучаем модель
rfr_best_params.fit(X_train, y_train)
# предсказания на test 
y_pred_rfr_best_params = rfr_best_params.predict(X_test)

In [113]:
# просмотр метрик модели RandomForestRegressor обученной на train с лучшими параметрами
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred = y_pred_rfr_best_params,
                           X_test = X_test,
                           name='RandomForestRegressor_best_params')])
metrics

,model,MAE,MSE,RMSE,RMSLE,R2 adjusted,MPE_%,MAPE_%,WAPE_%
0,RandomForestRegressor_Baseline,6.336102,268.425542,16.383697,None,0.888419,-inf,inf,NaN
0,DecisionTreeRegressor_Baseline,7.378031,513.967447,22.670850,None,0.781858,-inf,inf,NaN
0,GradientBoostingRegressor_Baseline,9.747325,429.918285,20.734471,None,0.827049,-inf,inf,NaN
0,RandomForestRegressor_best_params,6.666587,274.464284,16.566964,None,0.885145,-inf,inf,NaN


## DecisionTreeRegressor

In [114]:
# сетка параметров
param_grid = {
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}

# расчет количества моделей
num_models = math.prod(len(values) for values in param_grid.values())

print(f"Количество моделей для обучения: {num_models}")

Количество моделей для обучения: 12


In [115]:
# модель DecisionTreeRegressor
dtr = DecisionTreeRegressor(random_state=RAND)

# модель RandomizedSearchCV и обучение по сетке
rsCV = RandomizedSearchCV(dtr, param_grid, cv=3, n_iter=20, n_jobs=-1, random_state=RAND)
rsCV.fit(X_train, y_train)

c:\Users\myxa-\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 12 is smaller than n_iter=20. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=10),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'max_depth': [None, 5, 10],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5]},
                   random_state=10)

In [116]:
# просмотр параметров
best_params_dtr = rsCV.best_params_
print(best_params_dtr)
print(rsCV.best_score_)

{'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 10}
0.8187965860632777


In [117]:
# модель DecisionTreeRegressor лучшими параметрами
dtr_best_params = DecisionTreeRegressor(**best_params_dtr, random_state=RAND)
# обучаем модель
dtr_best_params.fit(X_train, y_train)
# предсказания на test 
y_pred_dtr_best_params = dtr_best_params.predict(X_test)

In [118]:
# просмотр метрик модели DecisionTreeRegressor обученной на train с лучшими параметрами
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred = y_pred_dtr_best_params,
                           X_test = X_test,
                           name='DecisionTreeRegressor_best_params')])
metrics

,model,MAE,MSE,RMSE,RMSLE,R2 adjusted,MPE_%,MAPE_%,WAPE_%
0,RandomForestRegressor_Baseline,6.336102,268.425542,16.383697,None,0.888419,-inf,inf,NaN
0,DecisionTreeRegressor_Baseline,7.378031,513.967447,22.670850,None,0.781858,-inf,inf,NaN
0,GradientBoostingRegressor_Baseline,9.747325,429.918285,20.734471,None,0.827049,-inf,inf,NaN
0,RandomForestRegressor_best_params,6.666587,274.464284,16.566964,None,0.885145,-inf,inf,NaN
0,DecisionTreeRegressor_best_params,7.474148,354.295108,18.822728,None,0.829074,-inf,inf,NaN


## GradientBoostingRegressor

In [121]:
# сетка параметров
param_grid = {
    'estimator__learning_rate': [0,1, 0,5, 1],
    'estimator__n_estimators': [100, 200, 300, 400, 500],
}

# расчет количества моделей
num_models = math.prod(len(values) for values in param_grid.values())

print(f"Количество моделей для обучения: {num_models}")

Количество моделей для обучения: 25


In [122]:
# модель GradientBoostingRegressor
gbr = MultiOutputRegressor(GradientBoostingRegressor(random_state=RAND))

# модель RandomizedSearchCV и обучение по сетке
rsCV = RandomizedSearchCV(gbr, param_grid, cv=3, n_iter=20, n_jobs=-1, random_state=RAND)
rsCV.fit(X_train, y_train)

c:\Users\myxa-\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [ 8.69637049e-001 -2.93889597e-005 -2.93889597e-005 -2.93889597e-005
 -2.93889597e-005             -inf -2.93889597e-005  8.68213313e-001
 -2.93889597e-005  8.68213313e-001 -2.93889597e-005  8.66813333e-001
  8.68936973e-001             -inf -2.93889597e-005  8.68936973e-001
  8.64368181e-001  8.66813333e-001  8.69637049e-001 -4.69197312e+240]
  warnings.warn(
c:\Users\myxa-\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:1113: RuntimeWarning: invalid value encountered in subtract
  (array - array_means[:, np.newaxis]) ** 2, axis=1, weights=weights
c:\Users\myxa-\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:1113: RuntimeWarning: overflow encountered in square
  (array - array_means[:, np.newaxis]) ** 2, axis=1, weights=weig

RandomizedSearchCV(cv=3,
                   estimator=MultiOutputRegressor(estimator=GradientBoostingRegressor(random_state=10)),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'estimator__learning_rate': [0, 1, 0, 5,
                                                                     1],
                                        'estimator__n_estimators': [100, 200,
                                                                    300, 400,
                                                                    500]},
                   random_state=10)

In [ ]:
# просмотр параметров
best_params_gbr = rsCV.best_params_
print(best_params_gbr)
print(rsCV.best_score_)

In [ ]:
# модель GradientBoostingRegressor лучшими параметрами
gbr_best_params = MultiOutputRegressor(GradientBoostingRegressor(**best_params_dtr, random_state=RAND))
# обучаем модель
gbr_best_params.fit(X_train, y_train)
# предсказания на test 
y_pred_gbr_best_params = gbr_best_params.predict(X_test)

In [ ]:
# просмотр метрик модели GradientBoostingRegressor обученной на train с лучшими параметрами
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred = y_pred_gbr_best_params,
                           X_test = X_test,
                           name='GradientBoostingRegressor_best_params')])

In [ ]:
#X = scaler.inverse_transform(X_scaled)

In [ ]:
# декодирование колонки 'country'
#X_train['country'] = le.inverse_transform(X_train['country'])
#X_test['country'] = le.inverse_transform(X_test['country'])